## Inference with PEFT

### Loading a Saved PEFT Model

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from peft import AutoPeftModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
lora_model = AutoPeftModelForCausalLM.from_pretrained("gpt2-lora")

### Generating Text from a PEFT Model

In [11]:
inputs = tokenizer("Hello, my name is ", return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=20)
print(tokenizer.batch_decode(outputs))

lora_outputs = lora_model.generate(input_ids=inputs["input_ids"], max_new_tokens=20)
print(tokenizer.batch_decode(lora_outputs))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Hello, my name is _____. I am a student at the University of California, Berkeley. I am a member of the']
['Hello, my name is !!!!!!!!!!!!!!!!!!!!']
